In [1]:
import numpy as np
import scipy.linalg as sla
import numpy.linalg as la
from scipy.special import expit as sigmoid
import scipy as sc
from statsmodels.tsa.api import VAR

In [2]:
varables = 3
observations = 1400
window = 2
#to use later 
times = 5

In [3]:
def qDag(w):
    return np.trace(sc.linalg.expm(np.multiply(w,w))) - w.shape[0]
def get_r(off):
    return lambda n: np.random.rand(n)+off

In [4]:
dagY = sc.sparse.random(varables, varables*window, density=0.15, data_rvs=get_r(.5)).A

dagX = np.eye(varables, varables)
while qDag(dagX) != 0:
    dagX = sc.sparse.random(varables, varables, density=0.25, data_rvs=get_r(.5)).A
    np.fill_diagonal(dagX, 0)

In [5]:
print(qDag(dagX))
print(dagX)
print(dagY)

0.0
[[0.         0.78169833 0.74042495]
 [0.         0.         0.        ]
 [0.         0.         0.        ]]
[[0.         0.         0.         0.         1.37394212 0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.66403981 0.         0.         0.         0.         0.59730823]]


In [6]:
def rand(n):
    return 2*(np.random.rand(n)-.5)

Y_pre = rand(observations*varables*window).reshape(varables*window, observations)
print(dagY.shape)
print(Y_pre.shape)

(3, 6)
(6, 1400)


In [7]:
#X = XW + YA + Z

def randFlat(X):
    return rand(X.shape[0]*X.shape[1]).reshape(*X.shape)

def onesFlat(X):
    return np.ones(X.shape[0]*X.shape[1]).reshape(*X.shape)

def update(preY, preX, dagY, dagX, rand):
    Xy = np.matmul(dagY, preY)
    Xr = rand(preX)
    Xx = np.matmul(dagX, Xr)
    X = Xr + Xx + Xy
    
    return X

def step(preY, dagY, dagX, window, var, rand):
    preX = preY[:var]
    preY = preY[:window*var]
    return update(preY, preX, dagY, dagX, rand)

def makeSynthData(preY, times, obs, var, pad, rand=rand):
    window = preY.shape[0]//var
    
    for i in range(pad+window):
        newX = step(preY, dagY, dagX, window, var, rand)
        preY = np.append(newX, preY, axis=0)

    Y = preY[:window*var]

    for i in range(times - window):
        newX = step(Y, dagY, dagX, window, var, rand)
        Y = np.append(newX, Y, axis=0)
        
    return Y
    



#later
#W = np.random.rand(varables*observations).reshape(varables,observations)

In [8]:
Y_pre = np.ones(observations*varables*window).reshape(varables*window, observations)
XY = makeSynthData(Y_pre, times, observations, varables, 0, randFlat)
Y=XY[varables:window*varables+varables]
X=XY[:varables]

In [9]:
XY

array([[-1.46905689, -0.23326422, -0.42714062, ...,  0.43946092,
        -1.20710198,  0.52168248],
       [ 0.06666353,  0.01504767,  0.5146201 , ...,  0.51459303,
        -0.08464904,  0.77914589],
       [ 0.42266355,  2.24891201,  1.58188714, ...,  2.36977383,
        -1.53556536,  1.20497474],
       ...,
       [ 1.48536423,  1.30949363,  1.85830257, ...,  3.01998084,
         2.12062303,  2.64484354],
       [ 0.75312325, -0.99685789, -0.10862971, ...,  0.56686016,
         0.21782882,  0.38265453],
       [ 1.94580828,  1.14664359,  1.74432331, ...,  2.12668207,
         1.23591904,  2.06676015]])

In [10]:
C_init = np.random.rand(varables*varables*window + varables*varables)

In [11]:
def f_DYNOTEARS(X, Y, lA, lB):
    def aux(C):
        a = C[:varables*window*varables].reshape(varables, varables*window)
        w = C[varables*window*varables:].reshape(varables, varables)
    
        loss = np.square(
                np.linalg.norm(
                    np.matmul(np.eye(varables, varables) - w, X)
                    - np.matmul(a, Y)
                    , "fro"
                )
            )/np.prod(X.shape)
        #print(loss)
        return loss + lB*np.linalg.norm(w.flatten(), 1) + lA*np.linalg.norm(a.flatten(), 1) 
    return aux

In [12]:
def constr_DYNOTEARS(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return qDag(w)

cons = {'type':'eq', 'fun': constr_DYNOTEARS}

C = sc.optimize.minimize(f_DYNOTEARS(X,Y,.1,.1) , C_init, constraints=cons).x

In [13]:
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)
np.fill_diagonal(W, 0)

In [14]:
A.round(1)

array([[-0. , -0. ,  0. ,  0. ,  0.9,  0. ],
       [-0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.6,  0. , -0. , -0. ,  0. ,  0.6]])

In [15]:
dagY.round(1)

array([[0. , 0. , 0. , 0. , 1.4, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. ],
       [0.7, 0. , 0. , 0. , 0. , 0.6]])

In [16]:
W.round(1)

array([[0. , 0.3, 0. ],
       [0. , 0. , 0. ],
       [0.1, 0. , 0. ]])

In [17]:
dagX.round(1)

array([[0. , 0.8, 0.7],
       [0. , 0. , 0. ],
       [0. , 0. , 0. ]])

In [18]:
dagX.T.round(1)

array([[0. , 0. , 0. ],
       [0.8, 0. , 0. ],
       [0.7, 0. , 0. ]])

In [19]:
def H1ldet(W):
    det = np.linalg.det(np.eye(W.shape[0]) - np.multiply(W,W))
    if (det <= 0):
        print(det)
    return -np.log(det)

def constr_DAGMA(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return H1ldet(w)

A_init = np.random.rand(varables*varables*window)

W_init = np.random.rand(varables*varables).reshape(varables, varables)
np.fill_diagonal(W_init, 0)
while H1ldet(W_init) is np.nan:
    W_init = np.random.rand(varables*varables).reshape(varables, varables)
    np.fill_diagonal(W, 0)

C_init = np.append(A_init, W_init.reshape(varables*varables), axis = 0)

cons = {'type':'eq', 'fun': constr_DAGMA}

#C = sc.optimize.minimize(f_DYNOTEARS(X,Y,.01,.1) , C_init, constraints=cons).x

In [20]:
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)
np.fill_diagonal(W, 0)

In [21]:
A[A<.3] = 0
A.round()

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1.]])

In [22]:
dagY.round()

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1.]])

In [23]:
W.round(1)

array([[0. , 0.3, 0. ],
       [0. , 0. , 0. ],
       [0.1, 0. , 0. ]])

In [24]:
dagX.round(1)

array([[0. , 0.8, 0.7],
       [0. , 0. , 0. ],
       [0. , 0. , 0. ]])

In [25]:
dagX.T.round(1)

array([[0. , 0. , 0. ],
       [0.8, 0. , 0. ],
       [0.7, 0. , 0. ]])

In [26]:
C_init = np.random.rand(varables*varables*window + varables*varables)
H1ldet(C_init.T)

-6.717066883840092


/var/folders/z_/p0tz320d6j13q0rkv_f35brm0000gn/T/ipykernel_32912/3164180601.py:5: RuntimeWarning: invalid value encountered in log
  return -np.log(det)


nan

In [27]:
def metrics(A,dagY, W, dagX):
    A = np.absolute(A) > 0.3
    dagY = np.absolute(dagY) > .5
    W = np.absolute(W) > 0.3
    dagX = np.absolute(dagX) > .5
    
    preWrong = A != dagY
    wWrong = W != dagX
    #assumes cant be both wrong and inv
    wInverse = W.T & dagX
    
    if ((W + W.T)==2).any():
        print(W)
        print(W.T)
        raise "error"
    
    #print(A)
    #print(dagY)
    #print(W)
    #print(dagX)
    #print()
    #print(preWrong)
    #print(wWrong)
    #print(wInverse)
    
    return {
        "preWrong": sum(sum(preWrong)),
        "wWrong": sum(sum(wWrong)),
        "wInverse": sum(sum(wInverse))
    }
    

In [28]:
metrics(A,dagY, W, dagX)

{'preWrong': 0, 'wWrong': 1, 'wInverse': 0}

In [29]:
A.round()

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1.]])

In [30]:
dagY.round()

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1.]])

In [31]:
np.matmul(np.eye(varables, varables) - W, X).round(1)

array([[-1.5, -0.2, -0.6, ...,  0.3, -1.2,  0.3],
       [ 0.1,  0. ,  0.5, ...,  0.5, -0.1,  0.8],
       [ 0.5,  2.3,  1.6, ...,  2.3, -1.5,  1.2]])

In [32]:
def transformTimeData(data, window, times):
    variables = data.shape[0]//times
    print(data.shape[0]/times)
    
    snapshots = times - window - 1
    
    ret = []
    
    for i in range(snapshots):
        slic = data[i*variables:i*variables+(window+1)*variables]
        ret.append( slic)
    
    return np.hstack(ret)
    
    

In [33]:
XY_stack = transformTimeData(XY, window, times)

3.0


In [34]:
Y_stack=XY_stack[varables:window*varables+varables]
X_stack=XY_stack[:varables]


In [35]:
print(XY.shape)
print(XY_stack.shape)
print(Y_stack.shape)
print(X_stack.shape)

(15, 1400)
(9, 2800)
(6, 2800)
(3, 2800)


In [36]:
XY.shape

(15, 1400)

In [37]:
XY_stack

array([[-1.46905689, -0.23326422, -0.42714062, ..., -0.4554368 ,
        -1.74059085, -0.45611304],
       [ 0.06666353,  0.01504767,  0.5146201 , ..., -0.17363273,
         0.6041638 ,  0.00400992],
       [ 0.42266355,  2.24891201,  1.58188714, ...,  3.60374725,
        -0.65012046,  1.17709002],
       ...,
       [-0.08852105, -0.52721938,  0.55847251, ...,  2.11521433,
         0.96789088,  0.37005571],
       [-0.56495309, -0.72715954, -0.46436555, ..., -0.27017328,
        -0.72825716, -0.89380375],
       [ 0.847856  ,  2.10872291,  2.19837723, ...,  3.18275944,
         1.70916549,  1.75910895]])

In [38]:
X_stack

array([[-1.46905689, -0.23326422, -0.42714062, ..., -0.4554368 ,
        -1.74059085, -0.45611304],
       [ 0.06666353,  0.01504767,  0.5146201 , ..., -0.17363273,
         0.6041638 ,  0.00400992],
       [ 0.42266355,  2.24891201,  1.58188714, ...,  3.60374725,
        -0.65012046,  1.17709002]])

In [39]:
Y_stack

array([[-0.47407867,  0.14961213,  0.46389201, ...,  2.05333973,
        -1.64873227, -1.14649429],
       [ 0.10304418, -0.07680757, -0.75650586, ..., -0.232362  ,
        -0.78115275, -0.62952893],
       [ 0.14501786, -0.11849988,  2.40090616, ...,  3.5142154 ,
         0.42476764,  1.20182644],
       [-0.08852105, -0.52721938,  0.55847251, ...,  2.11521433,
         0.96789088,  0.37005571],
       [-0.56495309, -0.72715954, -0.46436555, ..., -0.27017328,
        -0.72825716, -0.89380375],
       [ 0.847856  ,  2.10872291,  2.19837723, ...,  3.18275944,
         1.70916549,  1.75910895]])

In [40]:
cons = {'type':'eq', 'fun': constr_DYNOTEARS}
C = sc.optimize.minimize(f_DYNOTEARS(X_stack,Y_stack,.01,.1) , C_init, constraints=cons).x

In [41]:
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)
np.fill_diagonal(W, 0)

In [42]:
abs(A.round(1))

array([[0.1, 0. , 0. , 0. , 1.3, 0.1],
       [0. , 0. , 0. , 0. , 0. , 0. ],
       [0.6, 0. , 0. , 0. , 0. , 0.6]])

In [43]:
dagY.round(1)

array([[0. , 0. , 0. , 0. , 1.4, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. ],
       [0.7, 0. , 0. , 0. , 0. , 0.6]])

In [44]:
W.round(2)

array([[ 0.  ,  0.35,  0.2 ],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  , -0.  ,  0.  ]])

In [45]:
dagX.round(1)

array([[0. , 0.8, 0.7],
       [0. , 0. , 0. ],
       [0. , 0. , 0. ]])

In [46]:
dagX.T.round(1)

array([[0. , 0. , 0. ],
       [0.8, 0. , 0. ],
       [0.7, 0. , 0. ]])

In [47]:
#tensor representation for timeseries
dagYtesnor = np.transpose(np.array([dagY[:,i*varables:(i+1)*varables] for i in range(window)]),(0,2,1))

dagXtesnor = dagX
print(dagYtesnor.round().shape)

XYtesnor = XY.reshape(times, varables, observations)
print(XYtesnor.shape)

(2, 3, 3)
(5, 3, 1400)


In [48]:
XYtesnor

array([[[-1.46905689, -0.23326422, -0.42714062, ...,  0.43946092,
         -1.20710198,  0.52168248],
        [ 0.06666353,  0.01504767,  0.5146201 , ...,  0.51459303,
         -0.08464904,  0.77914589],
        [ 0.42266355,  2.24891201,  1.58188714, ...,  2.36977383,
         -1.53556536,  1.20497474]],

       [[-0.47407867,  0.14961213,  0.46389201, ..., -0.4554368 ,
         -1.74059085, -0.45611304],
        [ 0.10304418, -0.07680757, -0.75650586, ..., -0.17363273,
          0.6041638 ,  0.00400992],
        [ 0.14501786, -0.11849988,  2.40090616, ...,  3.60374725,
         -0.65012046,  1.17709002]],

       [[-0.08852105, -0.52721938,  0.55847251, ...,  2.05333973,
         -1.64873227, -1.14649429],
        [-0.56495309, -0.72715954, -0.46436555, ..., -0.232362  ,
         -0.78115275, -0.62952893],
        [ 0.847856  ,  2.10872291,  2.19837723, ...,  3.5142154 ,
          0.42476764,  1.20182644]],

       [[ 0.6933287 ,  0.68755409,  1.35519416, ...,  2.11521433,
          

In [49]:
XY.reshape(times*varables, observations)

array([[-1.46905689, -0.23326422, -0.42714062, ...,  0.43946092,
        -1.20710198,  0.52168248],
       [ 0.06666353,  0.01504767,  0.5146201 , ...,  0.51459303,
        -0.08464904,  0.77914589],
       [ 0.42266355,  2.24891201,  1.58188714, ...,  2.36977383,
        -1.53556536,  1.20497474],
       ...,
       [ 1.48536423,  1.30949363,  1.85830257, ...,  3.01998084,
         2.12062303,  2.64484354],
       [ 0.75312325, -0.99685789, -0.10862971, ...,  0.56686016,
         0.21782882,  0.38265453],
       [ 1.94580828,  1.14664359,  1.74432331, ...,  2.12668207,
         1.23591904,  2.06676015]])

In [50]:
Y_stack=XY_stack[varables:window*varables+varables]
X_stack=XY_stack[:varables]

In [51]:
Y_stack

array([[-0.47407867,  0.14961213,  0.46389201, ...,  2.05333973,
        -1.64873227, -1.14649429],
       [ 0.10304418, -0.07680757, -0.75650586, ..., -0.232362  ,
        -0.78115275, -0.62952893],
       [ 0.14501786, -0.11849988,  2.40090616, ...,  3.5142154 ,
         0.42476764,  1.20182644],
       [-0.08852105, -0.52721938,  0.55847251, ...,  2.11521433,
         0.96789088,  0.37005571],
       [-0.56495309, -0.72715954, -0.46436555, ..., -0.27017328,
        -0.72825716, -0.89380375],
       [ 0.847856  ,  2.10872291,  2.19837723, ...,  3.18275944,
         1.70916549,  1.75910895]])

In [52]:
X_stack

array([[-1.46905689, -0.23326422, -0.42714062, ..., -0.4554368 ,
        -1.74059085, -0.45611304],
       [ 0.06666353,  0.01504767,  0.5146201 , ..., -0.17363273,
         0.6041638 ,  0.00400992],
       [ 0.42266355,  2.24891201,  1.58188714, ...,  3.60374725,
        -0.65012046,  1.17709002]])

In [53]:
def diff(X, w, a):
    X_check = X[:X.shape[0]-a.shape[0]]
    
    Xw = np.matmul(w, X_check)
    #print(a.shape)
    #print(X.shape[0]-window+1)
    Xa = np.array([np.tensordot(a, X[i:i+window], axes=((0,1),(0,1)))
                    for i in range(1, X.shape[0]-window+1)])
    #print(X_check.shape)
    #print(Xw.shape)
    #print(Xa.shape)
    
    return X_check - Xw - Xa

def f_DYNOTEARS_TENSOR(X, lA, lB):
    def aux(C):
        a = C[:varables*window*varables].reshape(window, varables, varables)
        w = C[varables*window*varables:].reshape(varables, varables)
    
        loss = np.square(diff(X, w, a)).sum().sum().sum()/np.prod(X.shape)
        #print(loss)
        return loss + lB*np.linalg.norm(w.flatten(), 1) + lA*np.linalg.norm(a.flatten(), 1) 
    return aux

def ll(X, Y, w, a):
    return np.square(
                np.linalg.norm(
                    np.matmul(np.eye(varables, varables) - w, X)
                    - np.matmul(a, Y)
                    , "fro"
                )
            )/np.prod(X.shape)

In [54]:
Y_stack=XY_stack[varables:window*varables+varables]
X_stack=XY_stack[:varables]

In [55]:
def constr_DYNOTEARS(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return qDag(w)

cons = {'type':'eq', 'fun': constr_DYNOTEARS}

C = sc.optimize.minimize(f_DYNOTEARS_TENSOR(XYtesnor,.05,.05) , C_init, constraints=cons).x

In [56]:
A = C[:varables*window*varables].reshape(window, varables, varables)
W = C[varables*window*varables:].reshape(varables, varables)
print(ll(X_stack, Y_stack, dagX, dagY))

#print(ll(XYtesnor, W, A))

print(np.square(diff(XYtesnor, dagX, dagYtesnor)).sum().sum().sum()/np.prod(XYtesnor.shape))

print(np.square(diff(XYtesnor, W, A)).sum().sum().sum()/np.prod(XYtesnor.shape))

0.7148535086622156
0.479309614204831
0.25197513628493445


In [64]:
W.round(1)

array([[ 0. ,  0.4,  0. ],
       [ 0. ,  0. , -0. ],
       [ 0.1,  0. ,  0. ]])

In [66]:
dagXtesnor.round(1)

array([[0. , 0.8, 0.7],
       [0. , 0. , 0. ],
       [0. , 0. , 0. ]])

(3, 3)

In [59]:
A.round(1)

array([[[-0. , -0. ,  0.6],
        [-0. , -0. ,  0. ],
        [ 0. ,  0. ,  0. ]],

       [[ 0. ,  0. , -0. ],
        [ 1. , -0. ,  0. ],
        [-0. , -0. ,  0.6]]])

In [60]:
dagYtesnor.round(1)

array([[[0. , 0. , 0.7],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [1.4, 0. , 0. ],
        [0. , 0. , 0.6]]])

In [61]:
XYtesnor[1:1+window].round()

array([[[-0.,  0.,  0., ..., -0., -2., -0.],
        [ 0., -0., -1., ..., -0.,  1.,  0.],
        [ 0., -0.,  2., ...,  4., -1.,  1.]],

       [[-0., -1.,  1., ...,  2., -2., -1.],
        [-1., -1., -0., ..., -0., -1., -1.],
        [ 1.,  2.,  2., ...,  4.,  0.,  1.]]])

In [62]:
dagYtesnor.round(1)

array([[[0. , 0. , 0.7],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [1.4, 0. , 0. ],
        [0. , 0. , 0.6]]])

In [63]:
np.tensordot(A, XYtesnor[1:1+window], axes=((0,1),(0,1))).round(1)

array([[-0.6, -0.7, -0.5, ..., -0.2, -0.8, -0.6],
       [ 0. , -0. ,  0. , ...,  0. , -0. , -0. ],
       [ 0.2,  1.3,  1.5, ...,  1.7, -0.8,  0.4]])

In [63]:
XYtesnor[0].round()
XYtesnor[1:1+window].round()

array([[[ 2.,  2.,  1., ...,  2.,  1.,  2.],
        [ 2.,  3.,  0., ...,  1.,  2.,  1.],
        [ 3.,  0.,  0., ...,  2.,  2.,  2.]],

       [[ 1.,  2.,  0., ...,  2.,  1., -0.],
        [ 1.,  3.,  0., ...,  1.,  0.,  1.],
        [ 0.,  1.,  2., ...,  3.,  1.,  2.]]])

In [64]:
#np.array([dagY[:,i*varables:(i+1)*varables] for i in range(window)]).round(1)

In [65]:
dagY.round(1)

array([[0. , 0. , 0. , 0. , 1.3, 0. ],
       [0. , 0. , 0. , 0. , 1.2, 0. ],
       [0. , 0. , 0. , 0. , 0. , 1.3]])

In [66]:
Y_stack.round()

array([[ 2.,  2.,  1., ...,  2.,  1., -0.],
       [ 2.,  3.,  0., ...,  1.,  0.,  1.],
       [ 3.,  0.,  0., ...,  3.,  1.,  2.],
       [ 1.,  2.,  0., ...,  0.,  2.,  2.],
       [ 1.,  3.,  0., ...,  2.,  2.,  1.],
       [ 0.,  1.,  2., ...,  1.,  2.,  2.]])

In [67]:
np.matmul(dagY, Y_stack).round()

array([[1., 3., 0., ..., 2., 2., 2.],
       [1., 3., 0., ..., 2., 2., 1.],
       [1., 1., 2., ..., 2., 2., 3.]])

In [68]:
X_stack.round()

array([[ 1.,  3.,  1., ...,  2.,  1.,  2.],
       [ 1.,  3., -0., ...,  1.,  2.,  1.],
       [ 2.,  1.,  3., ...,  2.,  2.,  2.]])

In [69]:
dagX.round(1)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [70]:
W.round(1)

array([[0. , 0. , 0. ],
       [0.1, 0. , 0. ],
       [0. , 0. , 0. ]])

In [71]:
def get_sparse(X):
    ret = []
    for i in range(len(X)):
        for j in range(len(X[i])):
            val = X[i,j]
            if not np.isnan(val):
                print(val)
                ret.append((i, j, val))
    return ret

constraint_matrix_X = np.array(
[
    [np.nan, np.nan, np.nan],
    [np.nan, np.nan, np.nan],
    [0, 1.1, np.nan],
])
X_set = np.logical_not(np.isnan(constraint_matrix_X)).sum().sum()
print(X_set)
X_sparce = get_sparse(constraint_matrix_X)

            
constraint_matrix_Y = np.array(
[
    [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    [np.nan, np.nan, np.nan, 6, np.nan, np.nan],
    [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
])
Y_set = np.logical_not(np.isnan(constraint_matrix_Y)).sum().sum()
print(Y_set)
Y_sparce = get_sparse(constraint_matrix_Y)


#C_init = np.random.rand(varables*varables*window + varables*varables)
#not create no C_init with XYset less variables and instread enforce in loss func
#to avoid unnessasary copying

2
0.0
1.1
1
6.0


In [72]:
def enforce_cons(X, cons):
    for (i, j, val) in cons:
        X[i][j] = val

def f_DYNOTEARS_CONSTRAINT(X, Y, lA, lB, X_cons, Y_cons):
    def aux(C):
        w = C[varables*window*varables:].reshape(varables, varables)
        enforce_cons(w, X_cons)
        a = C[:varables*window*varables].reshape(varables, varables*window)
        enforce_cons(a, Y_cons)
    
        loss = np.square(
                np.linalg.norm(
                    np.matmul(np.eye(varables, varables) - w, X)
                    - np.matmul(a, Y)
                    , "fro"
                )
            )/np.prod(X.shape)
        #print(loss)
        return loss + lB*np.linalg.norm(w.flatten(), 1) + lA*np.linalg.norm(a.flatten(), 1) 
    return aux

In [73]:
def constr_DYNOTEARS(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return qDag(w)

cons = {'type':'eq', 'fun': constr_DYNOTEARS}

C = sc.optimize.minimize(f_DYNOTEARS_CONSTRAINT(X,Y,.1,.1,X_sparce, Y_sparce) , C_init, constraints=cons).x

In [74]:

W = C[varables*window*varables:].reshape(varables, varables)
enforce_cons(W, X_sparce)
A = C[:varables*window*varables].reshape(varables, varables*window)
enforce_cons(A, Y_sparce)
np.fill_diagonal(W, 0)

In [75]:
enforce_cons(A, Y_sparce)
A.round(1)

array([[ 0. ,  0. ,  0. ,  0. ,  1.2,  0. ],
       [-0.4, -0.3, -0.4,  6. , -3.3, -0.4],
       [ 0. ,  0. ,  0. , -0. , -1.1,  1.1]])

In [76]:
dagY.round(1)

array([[0. , 0. , 0. , 0. , 1.3, 0. ],
       [0. , 0. , 0. , 0. , 1.2, 0. ],
       [0. , 0. , 0. , 0. , 0. , 1.3]])

In [77]:
W.round(1)

array([[ 0. ,  0.1,  0. ],
       [-0. ,  0. ,  0. ],
       [ 0. ,  1.1,  0. ]])

In [78]:
dagX.round(1)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [79]:
#testing confounded in Y

varables = 3
observations = 1000
window = 3
#to use later 
times = 5
C_init = np.random.rand(varables*varables*window + varables*varables)

In [80]:
dagY = sc.sparse.random(varables, varables*window, density=0.15, data_rvs=get_r(.5)).A

dagX = np.eye(varables, varables)
    while qDag(dagX) != 0:
    dagX = sc.sparse.random(varables, varables, density=0.25, data_rvs=get_r(.5)).A
    np.fill_diagonal(dagX, 0)

In [81]:
for i in range(len(dagY)):
    dagY[i][i] = .2

In [82]:
Y_pre = np.ones(observations*varables*window).reshape(varables*window, observations)
XY = makeSynthData(Y_pre, times, observations, varables, 0, randFlat)
Y=XY[varables:window*varables+varables]
X=XY[:varables]

In [83]:
def f_DYNOTEARS(X, Y, lA, lB):
    def aux(C):
        a = C[:varables*window*varables].reshape(varables, varables*window)
        w = C[varables*window*varables:].reshape(varables, varables)
    
        loss = np.square(
                np.linalg.norm(
                    np.matmul(np.eye(varables, varables) - w, X)
                    - np.matmul(a, Y)
                    , "fro"
                )
            )/np.prod(X.shape)
        #print(loss)
        return loss + lB*np.linalg.norm(w.flatten(), 1) + lA*np.linalg.norm(a.flatten(), 1) 
    return aux

In [84]:
cons = {'type':'eq', 'fun': constr_DYNOTEARS}

C = sc.optimize.minimize(f_DYNOTEARS(X,Y,.1,.1) , C_init, constraints=cons).x
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/linalg/_matfuncs.py:378: RuntimeWarning: overflow encountered in matmul
  eAw = eAw @ eAw


In [85]:
W.round(1)

array([[ 0. , -0. ,  0.6],
       [ 0. ,  0. ,  0. ],
       [ 0. ,  0.3,  0. ]])

In [86]:
dagX.round(1)

array([[0. , 0. , 1.4],
       [0. , 0. , 1.4],
       [0. , 0. , 0. ]])

In [87]:
A.round(1)

array([[-0. , -0. , -0. , -0. ,  0.5,  0. , -0. ,  0. ,  0. ],
       [ 1.4,  0.2,  0. ,  0. ,  0. , -0. ,  0. ,  0.2,  0.3],
       [ 0. , -0. ,  0. ,  0. , -0. ,  0. , -0. , -0. , -0. ]])

In [88]:
dagY.round(1)

array([[0.2, 0. , 0. , 0. , 0.6, 0. , 0. , 0. , 0. ],
       [1.4, 0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0.9],
       [0.5, 0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0. ]])